pytorch기반 간단한 lstm 모델입니다. 중간중간 불필요한 코드는 주석 처리 하였습니다.

## 라이브러리 로딩

In [3]:
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

import torch
import torch.nn as nn
import torch.optim as optim

### 예측해야될 데이터
---
- dangjin_floating : 당진수상태양광 발전량(KW)
- dangjin_warehouse : 당진자재창고태양광 발전량(KW)
- dangjin : 당진태양광 발전량(KW)
- ulsan : 울산태양광 발전량(KW)

In [10]:
energy = pd.read_csv('energy.csv')
dangjin_fcst = pd.read_csv('dangjin_fcst_data.csv')
ulsan_fcst = pd.read_csv('ulsan_fcst_data.csv')
pd.read_csv('dangjin_fcst_data.csv')

,Forecast time,forecast,Temperature,Humidity,WindSpeed,WindDirection,Cloud
0,2018-03-01 11:00:00,4.0,0.0,60.0,7.3,309.0,2.0
1,2018-03-01 11:00:00,7.0,-2.0,60.0,7.1,314.0,1.0
2,2018-03-01 11:00:00,10.0,-2.0,60.0,6.7,323.0,1.0
3,2018-03-01 11:00:00,13.0,-2.0,55.0,6.7,336.0,1.0
4,2018-03-01 11:00:00,16.0,-4.0,55.0,5.5,339.0,1.0
...,...,...,...,...,...,...,...
162203,2021-03-01 08:00:00,52.0,7.0,40.0,3.2,187.0,1.0
162204,2021-03-01 08:00:00,55.0,8.0,40.0,4.5,217.0,1.0
162205,2021-03-01 08:00:00,58.0,5.0,55.0,2.2,210.0,1.0
162206,2021-03-01 08:00:00,61.0,1.0,80.0,1.9,164.0,1.0


In [11]:
def to_date(x):
    return timedelta(hours=x)

In [12]:
def generate_df(df_):
    """
    20시 예보 데이터와 23시 예보 데이터만 활용하기 위해 데이터 추출
    """
    df = df_.copy()

    df['Forecast_time'] = pd.to_datetime(df['Forecast time'])

    # 20 시
    a = df[df["forecast"] == 4.0]
    a = a[a["Forecast time"].apply(lambda x: "20:00:00" in x)]
    a.loc[:, 'Forecast_time'] = a.loc[:, 'Forecast_time'] + \
        a.loc[:, 'forecast'].map(to_date)

    # 23 시
    b = df[df["forecast"] <= 22]
    b = b[b["Forecast time"].apply(lambda x: "23:00:00" in x)]
    b.loc[:, 'Forecast_time'] = b.loc[:, 'Forecast_time'] + \
        b.loc[:, 'forecast'].map(to_date)

    # 병합
    c = pd.concat([a, b])
    print(f"20시 사용 데이터 길이 : {len(a)}")
    print(f"23시 사용 데이터 길이 : {len(b)}")
    print(f"합친 데이터 길이 : {len(c)}")
    print()

    # 정렬
    c.sort_values(by=['Forecast_time'], inplace=True)
    c = c[['Forecast_time', 'Temperature', 'Humidity',
           'WindSpeed', 'WindDirection', 'Cloud']]

    return c

In [19]:
def interpolate_df(df, method='linear'):
    """
    결측치를 보간법으로 채워주는함수 
    """
    new_df = pd.DataFrame()
    new_df['Forecast_time'] = pd.date_range(
        start=df['Forecast_time'].iloc[0], end=df['Forecast_time'].iloc[-1], freq='H')
    new_df = pd.merge(new_df, df, on='Forecast_time', how='outer')
    new_df['Forecast_time'] = new_df['Forecast_time'].astype('str')
    return new_df.interpolate(method=method)

In [20]:
def train_datast(energy_df, fcst_df, target):
    """
    발전량과 일기예보 데이터 셋을 결합하여 train 데이터셋으로 만들어 주는 함수
    """
    # 일기 예보 있는 날짜만 선택
    energy = energy_df.loc[24:]
    energy.index = range(energy.shape[0])

    # 발전량 데이터가 있는 날짜만 선택
    fcst = fcst_df
    fcst.index = range(fcst.shape[0])

    # 발전량과 일기예보 연결
    concat_df = pd.concat([energy, fcst], axis=1)

    # 예보 시간, 날짜, 기상 예보 및 발전량 선택
    feature_df = concat_df[['Temperature', 'Humidity',
                            'WindSpeed', 'WindDirection', 'Cloud', target]]
    feature_df.fillna(0, inplace=True)

    return np.array(feature_df[:-22])

In [ ]:
dangjin_filled = generate_df(dangjin_fcst)
ulsan_filled = generate_df(ulsan_fcst)

In [21]:
dangjin_interpolated = interpolate_df(dangjin_filled, method='ffill')
ulsan_interpolated = interpolate_df(ulsan_filled, method='ffill')

In [ ]:
dangjin_floating_df = train_datast(
    energy, dangjin_interpolated, target='dangjin_floating')
ulsan_df = train_datast(energy, ulsan_interpolated, target='ulsan')
dangjin_df = train_datast(energy, dangjin_interpolated, target='dangjin')
dangjin_warehouse_df = train_datast(
    energy, dangjin_interpolated, target='dangjin_warehouse')

In [23]:
from sklearn.model_selection import train_test_split


def make_batch(input_data, sl=24):
    """
    energy 데이터를 sequence length길이에 맞춰 input형태로 변환 시 켜준다.
    그리고 train데이터 셋과 test데이터셋을 나눈다.

    파라미터 
    ---
    input_data : 
        energy 데이터
    sl : int
        sequence length 

    returns
    train_x : Tensor
        model의 학습용 input data
    train_y : Tensor
        model의 학습용 target data    
    valid_x : Tensor
        model의 검증용 input data
    valid_y : Tensor
        model의 검증용 target data    
    """
    train_x = []
    train_y = []
    L = len(input_data)
    for i in range(L-sl):
        train_seq = input_data[i:i+sl, :]
        train_label = input_data[i+sl, [-1]]
        train_x.append(train_seq)
        train_y.append(train_label)

    tensor_x, tensor_y = torch.Tensor(train_x), torch.Tensor(train_y)
    train_x, valid_x, train_y, valid_y = train_test_split(
        tensor_x, tensor_y, test_size=0.3)

    return train_x, valid_x, train_y, valid_y

In [24]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader


class CustomDataset(torch.utils.data.Dataset):
    """
    mini batch 학습을 위한 customdataset
    """

    def __init__(self, tensor_x, tensor_y):
        self.x_data = tensor_x
        self.y_data = tensor_y

    def __len__(self):
        return len(self.x_data)

    def __getitem__(self, idx):
        x = self.x_data[idx]
        y = self.y_data[idx]
        return x, y

In [25]:
# 모델 설계
class simple_lstm(nn.Module):

    def __init__(self, input_vector, sl, output_vector, num_layers):
        super().__init__()
        self.input_vector = input_vector
        self.sequence_length = sl
        self.output_vector = output_vector
        self.num_layers = num_layers

        self.lstm = nn.LSTM(input_size=self.input_vector, hidden_size=self.output_vector,
                            num_layers=self.num_layers, batch_first=True)
        self.linear = nn.Sequential(
            nn.Linear(self.output_vector, 50),
            nn.Linear(50, 30),
            nn.Linear(30, 10),
            nn.Linear(10, 1)
        )

    def forward(self, x):
        output, _ = self.lstm(x)  # (hidden, cell) 데이터는 사용하지 않음
        return self.linear(output[:, -1, :])


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [26]:
def training(model, EPOCHS, optimizer, criterion, train_loader, valid_loader):
    """
    model을 학습하는함수 
    검증데이터셋을 이용하여 가장 성능이 좋은 모델을 반환한다.
    """
    best_model = model
    BEST_LOSS = int(1e9)
    for i in range(EPOCHS):

        TRAIN_LOSS = 0
        VALID_LOSS = 0
        for batch_idx, batch in enumerate(train_loader):
            model.train()
            tensor_x, tensor_y = batch
            # optimizer 초기화
            optimizer.zero_grad()

            tensor_x = tensor_x.to(device)
            tensor_y = tensor_y.to(device)
            output = model(tensor_x)
            loss = criterion(output, tensor_y.view(-1, 1))

            # loss 누적
            TRAIN_LOSS += loss.item()

            loss.backward()
            optimizer.step()

        model.eval()

        for batch_idx, batch in enumerate(valid_loader):
            tensor_x, tensor_y = batch
            with torch.no_grad():
                tensor_x = tensor_x.to(device)
                tensor_y = tensor_y.to(device)
                output = model(tensor_x)
                loss = criterion(output, tensor_y.view(-1, 1))
                VALID_LOSS += loss.item()

        # best 모델 저장
        if VALID_LOSS < BEST_LOSS:
            best_model = model
            BEST_LOSS = VALID_LOSS

        if i % 100 == 0:
            print('Epoch {}, train_Loss {:.5f}, valid_Loss {:.5f}'.format(
                i, TRAIN_LOSS, VALID_LOSS))

    return best_model

In [27]:
def inference(df, model):
    """
    학습된 모델을 이용하여 발전소별 발전량을 예측하는 함수
    """

    x_input = np.array(df[-48:])  # next value based on data of last year
    x_input = x_input.reshape((1, 48, 1))
    model_pred = []

    for i in range(672):

        x_input = torch.Tensor(x_input)
        x_input = x_input.to(device)
        predict = dangjin_floating(x_input).cpu().detach().numpy()

        new_input = predict.reshape((1, 1, 1))
        x_input = np.concatenate((x_input[:, -47:].cpu(), new_input), axis=1)
        model_pred.append(predict[0][0])

### dangjin_floating

In [28]:
def train_and_inference(df):
    """
    각 발전소 별 모델 학습하고 
    성능이 가장 좋은 모델을 이용하여 추론한 결과를 반환하는 함수

    파라미터 
    ---
    df : dataframe
        발전소 별 발전량

    return
    ---
    pred : list
        학습된 모델을 이용한 예측값
    """

    # hyper parameters
    SEQUENCE_LENGTH = 48
    INPUT_VECTOR = 1
    OUTPUT_VECTOR = 100
    NUM_LAYERS = 4
    EPOCHS = 1000
    LR = 0.0001

    # 모델 선언
    lstm = simple_lstm(INPUT_VECTOR, SEQUENCE_LENGTH,
                       OUTPUT_VECTOR, NUM_LAYERS).to(device)

    optimizer = torch.optim.AdamW(lstm.parameters(), lr=LR)
    criterion = nn.MSELoss()

    train_x, valid_x, train_y, valid_y = make_batch(
        df.reshape(-1, 1), SEQUENCE_LENGTH)
    trn_data = CustomDataset(train_x, train_y)
    val_data = CustomDataset(valid_x, valid_y)
    train_loader = DataLoader(trn_data, batch_size=256, shuffle=True)
    valid_loader = DataLoader(val_data, batch_size=256, shuffle=True)
    print('--------------------train start--------------------')
    best_model = training(lstm, EPOCHS, optimizer,
                          criterion, train_loader, valid_loader)
    print('--------------------inference start--------------------')
    pred = inference(df, best_model)

    return pred

In [ ]:
dangjin_floating_pred = train_and_inference(dangjin_floating_df)
ulsan_pred = train_and_inference(ulsan_df)
dangjin_warehouse_pred = train_and_inference(dangjin_warehouse_df)
dangjin_pred = train_and_inference(dangjin_df)

### 정답 파일 생성

In [ ]:
submission = pd.read_csv('submission.csv')

In [ ]:
submission.iloc[:24*28, 1] = dangjin_floating_pred
submission.iloc[:24*28, 2] = dangjin_warehouse_pred
submission.iloc[:24*28, 3] = dangjin_pred
submission.iloc[:24*28, 4] = ulsan_pred

In [ ]:
submission.to_csv('submission.csv', index=False)

In [ ]:
submission